In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [3]:
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0})

In [4]:
len(final_documents)

316

In [5]:
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

e:\MLProject\LLM\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [6]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.46568570e-02 -1.19099235e-02 -3.37892473e-02  2.94559356e-02
  5.19159846e-02  5.73839620e-02 -4.10017557e-02  2.74268296e-02
 -1.05128199e-01 -1.58056319e-02  7.94858783e-02  5.64318672e-02
 -1.31765315e-02 -3.41544002e-02  5.81603544e-03  4.72547635e-02
 -1.30746923e-02  3.12991021e-03 -3.44226211e-02  3.08406539e-02
 -4.09086384e-02  3.52738090e-02 -2.43761279e-02 -4.35831621e-02
  2.41503250e-02  1.31986430e-02 -4.84449696e-03  1.92347690e-02
 -5.43913022e-02 -1.42735079e-01  5.15529374e-03  2.93115694e-02
 -5.60810864e-02 -8.53535626e-03  3.14140990e-02  2.76736431e-02
 -2.06188150e-02  8.24231431e-02  4.15425450e-02  5.79655133e-02
 -3.71587276e-02  6.26160763e-03 -2.41390169e-02 -5.61793987e-03
 -2.51715239e-02  5.04972134e-03 -2.52800807e-02 -2.91946088e-03
 -8.24046507e-03 -5.69604412e-02  2.30822824e-02 -5.54221077e-03
  5.11555597e-02  6.09937571e-02  6.49765953e-02 -5.38514033e-02
  2.19109822e-02 -2.54194103e-02 -4.49223220e-02  4.22458798e-02
  4.75251973e-02  7.23213

In [7]:
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [8]:
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [9]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018283709790> search_kwargs={'k': 3}


In [37]:
from langchain_huggingface import HuggingFaceEndpoint

In [38]:
import os
HUGGINGFACEHUB_API_TOKEN=os.environ.get('HUGGINGFACEHUB_API_TOKEN')

In [46]:
repo_id="mistralai/Mistral-7B-Instruct-v0.2"
hf=HuggingFaceEndpoint(repo_id=repo_id,max_length=128,temperature=0.7,token=HUGGINGFACEHUB_API_TOKEN)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


In [47]:

query="What is the health insurance coverage?"
hf.invoke(query)

"\n\nHealth insurance is a type of insurance that covers the cost of medical and surgical expenses. It is a contract between an insurance company and an individual or group, in which the insurance company agrees to pay some or all of the insured's healthcare expenses in exchange for a premium.\n\nHealth insurance can help protect against financial losses due to unexpected medical expenses, and can provide peace of mind knowing that healthcare costs are covered. It can also help individuals and families access necessary medical care and services that they may not be able to afford otherwise.\n\nThere are different types of health insurance plans, including:\n\n1. Fee-for-service plans: In this type of plan, the insured pays a set amount for each healthcare service they receive.\n2. Health maintenance organization (HMO) plans: In an HMO, the insured chooses a primary care physician who coordinates all of their healthcare needs. The insured must generally receive care from providers withi

In [48]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

In [49]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [50]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [51]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [52]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [53]:
result = retrievalQA.invoke({"query": query})
print(result['result'])

1. The uninsured rate in 2022 ranged from a low of 2.4% in Massachusetts to a high of 16.6% in Texas, with a national rate of 8.0%.
 2. Medicaid coverage accounted for a larger percentage in states that expanded Medicaid eligibility (22.7%) compared to nonexpansion states (18.0%).
 3. From 2021 to 2022, uninsured rates decreased in 27 states and increased in Maine, but Maine's rate remained below the national average.
 4. Private coverage rates were not statistically different in North Dakota and Utah.
 5. The most populous metropolitan areas with the lowest uninsured rates in 2022 were Boston-Cambridge-Newton/comma.tab MA-NH, San Francisco-Oakland-Berkeley/comma.tab CA, and Detroit-Warren-Dearborn/comma.tab MI.


In [41]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.

model_id="mistralai/Mistral-7B-Instruct-v0.2"
hf = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)
llm=hf
llm.invoke(query)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2.
403 Client Error. (Request ID: Root=1-66dfbe20-07b2b1ce37a9375e604d0af7;b858b96d-1b83-4aec-8e43-c0868f9518cd)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted and you are not in the authorized list. Visit https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2 to ask for access.